# Validate Python 'Heat Units' module

This notebook will use functions from the [HeatUnits](https://github.com/UCANR-IGIS/caladapt-py/blob/master/CookBooks/HeatUnits.py) Python module to compute degree days for a test data set from the [Degree Day Challenge](https://ucanr-igis.github.io/degree-day-challenge/). It will then compare the results with the reference answers from the UC IPM website. For more info on the datasets, see the [Degree Day Challenge](https://ucanr-igis.github.io/degree-day-challenge/).

<div style="width:80%; background-color:#eee; border:2px solid gray; margin:0 auto; padding:1em;">
<strong>Summary of results</strong>

- No differences were found between the computations and the reference answers for horizontal cutoffs.

- A small number (4-10) of days out of 365 had relatively big differences between the UC IPM website and `HeatU()` for the intermediate and vertical cutoffs. This was seen in all methods (single-triangle, single-sine, double-triangle, double-sine)
    
- Next steps: 1) repeat the calculations on the IPM website, 2) look for an alternative way (e.g., graphical) to compute the metric

</div>  

First we import the Pandas package and the HeatUnits module:

In [ ]:
import pandas as pd
import HeatUnits as hu

Next we import the reference temperature dataset:

In [ ]:
# Read CSV into pandas dataframe
data = pd.read_csv('espartoa-weather-2020.csv')

# Preview
pd.reset_option('display.max_rows')
data

Next, we add columns for the following day's min and max temperatures. Some of the degree day formula need these.

In [ ]:
# Add column for yesterdays min temp and fill the NA values by repeating the prev/next row
data["tmin_tom"] = data["tmin"].shift(-1)
data.tmin_tom.fillna(data.tmin, inplace=True)

# add column for tomorrows max temp
data["tmax_yest"] = data["tmax"].shift(1)
data.tmax_yest.fillna(data.tmax, inplace=True)

# Preview
data

The HeatUnits module has a single function `HeatU` that computes degree days from the daily minimum and maximum temperature using the methods described in Zalom (1983) and three different cutoff methods. 

Let's look at the help page for `HeatU`:

In [ ]:
help(hu.HeatU)

The arguments required include:
- `lthres` - lower temperature threshold
- `cthres` - upper temperature threshold
- `cm` - computation method (1 = single sine; 2 = double sine; 3 = single triangle, 4 = double triangle; 5 = Huber's method)
- `coff` - cuttoff method (1 = horizontal, 2 = intermediate, 3 = vertical)
- `ci` - Computation interval (ASK SHANE WHAT IS THIS ARGUMENT FOR ?????)
- `tmin` - minimum daily temp
- `tmax` - maximum daily temp
- `tmin_tomm` - tomorrow's daily minimum temp
- `tmax_yest` - yesterday's daily maximum temp

Note that several of the arguments expect a temperature value. You can use either Farenheit or Celcius, as long as you're consistent.

To make our live a little easier, we define some global constants:

In [ ]:
CM_SNGSIN = 1
CM_DBLSIN = 2
CM_SNGTRI = 3
CM_DBLTRI = 4
CM_HUBER = 5

COFF_H = 1
COFF_I = 2
COFF_V = 3                         

To demonstrate how to compute degree days with `HeatU()`, let's compute single sine method using a horizontal cutoff:

In [ ]:
LOWER_THRESH = 50
UPPER_THRESH = 70

data_sisine = data.copy()
data_sisine["SiSineHor"] = data.apply(lambda x: hu.HeatU(LOWER_THRESH, UPPER_THRESH, CM_SNGSIN, COFF_H, 1, 
                                                  x['tmin'], x['tmax']), axis=1)

data_sisine

# Import Reference Answers

To compare these calculations against the reference answers, we first import the 'correct' answers available from the [Degree Day Challenge](https://github.com/UCANR-IGIS/degree-day-challenge/blob/main/data/ucipm_results/ucipm_low50_high70_all.csv) website:

In [ ]:
# Read the CSV of official answers into pandas dataframe
ans = pd.read_csv('ucipm_low50_high70_all.csv')

# Preview
ans

## Compare Answers

To compare the values against reference answers, we i) round the values to two decimal places (because the reference values are rounded), and then subtract:

In [ ]:
sngsine_diff = data_sisine.round({'SiSineHor': 2})['SiSineHor'] - ans['sngsine_horiz']

View the results (they should all be zero):

In [ ]:
pd.set_option('display.max_rows', None)
display(sngsine_diff)

An easier way to view the differences is through a frequency table:

In [ ]:
sngsine_diff.value_counts()


# Compute and Compare Single Triangle Methods

Next we compute all the degree day methods, starting with the triangle methods:

In [ ]:
data_sngtri = data.copy()
data_sngtri["SiTriHor"] = data_sngtri.apply(lambda x: hu.HeatU(50, 70, CM_SNGTRI, COFF_H, 1, x['tmin'], x['tmax']), axis=1)
data_sngtri["SiTriInt"] = data_sngtri.apply(lambda x: hu.HeatU(50, 70, CM_SNGTRI, COFF_I, 1, x['tmin'], x['tmax']), axis=1)
data_sngtri["SiTriVert"] = data_sngtri.apply(lambda x: hu.HeatU(50, 70, CM_SNGTRI, COFF_V, 1, x['tmin'], x['tmax']), axis=1)
data_sngtri_cols = ['station', 'date', 'tmin', 'tmax', 'tmin_tom', 'tmax_yest', 'SiTriHor', 'SiTriInt', 'SiTriVert']

# data_sngtri = data_sngtri[data_sngtri_cols]      ## works but causes an error below
data_sngtri = data_sngtri.loc[:,data_sngtri_cols]  
data_sngtri.head()

Now compute the differences between the values computed by `HeatU()` and the answers from the IPM website:

In [ ]:
data_sngtri["SiTriHor_diff"] = round(data_sngtri.SiTriHor, 2) - ans['sngtri_horiz']
data_sngtri["SiTriInt_diff"] = round(data_sngtri.SiTriInt, 2) - ans['sngtri_intrmd']
data_sngtri["SiTriVert_diff"] = round(data_sngtri.SiTriVert, 2) - ans['sngtri_vert']
data_sngtri.head()

### Compare Differences: Single triangle with horizonal cutoff

In [ ]:
# Single triangle with horizonal cutoff
data_sngtri.SiTriHor_diff.value_counts()

### Compare Differences: Single triangle with intermediate cutoff

In [ ]:
# Single triangle with intermediate cutoff
data_sngtri.SiTriInt_diff.value_counts()

Oh dear. Let's review the rows with significant differences: 

In [ ]:
data_sngtri[data_sngtri.SiTriInt_diff >= 10]

### Compare Differences: Single triangle with vertical cutoff

In [ ]:
# Single triangle with vertical cutoff
data_sngtri.SiTriVert_diff.value_counts()

# Investigate Differences in the Single Triangle Methods

Oh dear, we see 4 days of the year where we're getting a big difference between the IPM website and the calculations from `HeatU()` for the single triangle method with the **intermediate** and **vertical** cutoffs (horizontal is fine).

Let's look at the rows where the difference is 20:

In [ ]:
data_sngtri[data_sngtri.SiTriVert_diff >= 20]

To see what's going on, let's join the `data_sngtri` and `ans` dataframes. They each have a 'date' column, however these columns are strings in different formats (see below for code to convert both of them to `datetime64[ns]`). So instead we'll merge them with `Dataframe.join()`.

## Join the Answer Table

Step 1 is to rename the columns in `data_sngtri` (to make it clear which dataframe they're from):

In [ ]:
data_sngtri.rename(columns={'date': 'date_hu', 'tmin': 'tmin_hu', 'tmax': 'tmax_hu',
                           'SiTriHor': 'SiTriHor_hu', 'SiTriInt': 'SiTriInt_hu', 
                           'SiTriVert': 'SiTriVert_hu'}, inplace=True)
data_sngtri.head()

Next, create a subset of the columns of the Answers dataframe.

In [ ]:
## Take a subset of columns from the reference dataset
ans_cols = ['date', 'tmin', 'tmax', 'sngtri_horiz', 'sngtri_intrmd', 'sngtri_vert']
ans_sngtri = ans.loc[:, ans_cols].copy()
ans_sngtri.head()

Rename them:

In [ ]:
## Rename some of the columns
ans_sngtri.rename(columns={'date': 'date_ipm', 'tmin': 'tmin_ipm', 'tmax': 'tmax_ipm',
                           'sngtri_horiz': 'SiTriHor_ipm', 'sngtri_intrmd': 'SiTriInt_ipm', 
                           'sngtri_vert': 'SiTriVert_ipm'}, inplace=True)
ans_sngtri.head()

Before we join them, let's verify they have the same number of rows:

In [ ]:
data_sngtri.shape
ans_sngtri.shape

Now we can join them:

In [ ]:
sngtri_comb = data_sngtri.join(ans_sngtri, lsuffix='_hu', rsuffix='_ipm')
sngtri_comb.head()

In [ ]:
sngtri_comb = sngtri_comb.reindex(columns=['station', 'date_hu', 'date_ipm', 'tmin_hu', 'tmin_ipm', 'tmax_hu',
                                           'tmax_ipm', 'tmin_tom', 'tmax_yest', 
                                           'SiTriHor_hu', 'SiTriHor_ipm', 'SiTriHor_diff',
                                           'SiTriInt_hu', 'SiTriInt_ipm', 'SiTriInt_diff', 
                                           'SiTriVert_hu', 'SiTriVert_ipm', 'SiTriVert_diff'])
sngtri_comb.head()

View the rows with the big differences:

In [ ]:
sngtri_comb[sngtri_comb.SiTriVert_diff >= 20]

# Single Sine Methods

In [ ]:
data_sngsin = data.copy()
data_sngsin["SiSineHor"] = data_sngsin.apply(lambda x: hu.HeatU(50, 70, CM_SNGSIN, COFF_H, 1, x['tmin'], x['tmax']), axis=1)
data_sngsin["SiSineInt"] = data_sngsin.apply(lambda x: hu.HeatU(50, 70, CM_SNGSIN, COFF_I, 1, x['tmin'], x['tmax']), axis=1)
data_sngsin["SiSineVert"] = data_sngsin.apply(lambda x: hu.HeatU(50, 70, CM_SNGSIN, COFF_V, 1, x['tmin'], x['tmax']), axis=1)
data_sngsin.head()

Compare with IPM answers:

In [ ]:
ans.head()

In [ ]:
data_sngsin.head()

In [ ]:
data_sngsin["SiSineHor_diff"] = round(data_sngsin.SiSineHor, 2) - ans['sngsine_horiz']
data_sngsin["SiSineInt_diff"] = round(data_sngsin.SiSineInt, 2) - ans['sngsine_intrmd']
data_sngsin["SiSineVert_diff"] = round(data_sngsin.SiSineVert, 2) - ans['sngsine_vert']
data_sngsin.head()

Look at the distribution of the differences:

In [ ]:
data_sngsin.SiSineHor_diff.value_counts()

In [ ]:
data_sngsin.SiSineInt_diff.value_counts()

In [ ]:
data_sngsin.SiSineVert_diff.value_counts()

See which rows are causing the issue:

In [ ]:
data_sngsin[data_sngsin.SiSineVert_diff > 10]

<p style="color:red; text-align:center; font-size:150%; line-height:30px;">There seem to be 4 rows where the results of `HeatU()` for the Single Sine method with the intermediate and vertical cutoffs that differ from those from the IPM website</p>

# Double Sine Methods

In [ ]:
data_dblsin = data.copy()
data_dblsin["DoSineHor"] = data_dblsin.apply(lambda x: hu.HeatU(50, 70, CM_DBLSIN, COFF_H, 1, x['tmin'], x['tmax'], x['tmin_tom'], x['tmax_yest']), axis=1)
data_dblsin["DoSineInt"] = data_dblsin.apply(lambda x: hu.HeatU(50, 70, CM_DBLSIN, COFF_I, 1, x['tmin'], x['tmax'], x['tmin_tom'], x['tmax_yest']), axis=1)
data_dblsin["DoSineVert"] = data_dblsin.apply(lambda x: hu.HeatU(50, 70, CM_DBLSIN, COFF_V, 1, x['tmin'], x['tmax'], x['tmin_tom'], x['tmax_yest']), axis=1)
data_dblsin.head()

Compute differences with the IPM answers:

In [ ]:
data_dblsin["DoSineHor_diff"] = round(data_dblsin.DoSineHor, 2) - ans['dblsine_horiz']
data_dblsin["DoSineInt_diff"] = round(data_dblsin.DoSineInt, 2) - ans['dblsine_intrmd']
data_dblsin["DoSineVert_diff"] = round(data_dblsin.DoSineVert, 2) - ans['dblsine_vert']
data_dblsin.head()

In [ ]:
## Double sine horizontal cutoff
data_dblsin.DoSineHor_diff.value_counts()

In [ ]:
## Double sine intermediate cutoff
data_dblsin.DoSineInt_diff.value_counts()

In [ ]:
## Double sine vertical cutoff
data_dblsin.DoSineVert_diff.value_counts()

Identify rows where they don't agree:

In [ ]:
data_dblsin[data_dblsin.DoSineVert_diff > 1]

In [ ]:
data_dblsin[data_dblsin.DoSineInt_diff > 1]

<p style="color:red; text-align:center; font-size:150%; line-height:30px;">There seem to be 7 rows where the results of `HeatU()` for the Double Sine method with the intermediate and vertical cutoffs that differ from those from the IPM website</p>

# Double-Triangle Methods

In [ ]:
data_dbltri = data.copy()
data_dbltri["DoTriHor"] = data_dbltri.apply(lambda x: hu.HeatU(50, 70, CM_DBLTRI, COFF_H, 1, x['tmin'], x['tmax'], x['tmin_tom'], x['tmax_yest']), axis=1)
data_dbltri["DoTriInt"] = data_dbltri.apply(lambda x: hu.HeatU(50, 70, CM_DBLTRI, COFF_I, 1, x['tmin'], x['tmax'], x['tmin_tom'], x['tmax_yest']), axis=1)
data_dbltri["DoTriVert"] = data_dbltri.apply(lambda x: hu.HeatU(50, 70, CM_DBLTRI, COFF_V, 1, x['tmin'], x['tmax'], x['tmin_tom'], x['tmax_yest']), axis=1)
data_dbltri.head()

Compute differences with IPM website:

In [ ]:
data_dbltri["DoTriHorr_diff"] = round(data_dbltri.DoTriHor, 2) - ans['dbltri_horiz']
data_dbltri["DoTriInt_diff"] = round(data_dbltri.DoTriInt, 2) - ans['dbltri_intrmd']
data_dbltri["DoTriVert_diff"] = round(data_dbltri.DoTriVert, 2) - ans['dbltri_vert']
data_dbltri.head()

Look at the distribution of the differences:

In [ ]:
## Double triangle horizontal cutoff
data_dbltri.DoTriHorr_diff.value_counts()

In [ ]:
## Double triangle intermediate cutoff
data_dbltri.DoTriInt_diff.value_counts()

In [ ]:
## Double triange vertical cutoff
data_dbltri.DoTriVert_diff.value_counts()

Investigate rows with the differences occur:

In [ ]:
data_dbltri[data_dbltri.DoTriVert_diff > 1]

In [ ]:
data_dbltri[data_dbltri.DoTriInt_diff > 1]

<p style="color:red; text-align:center; font-size:150%; line-height:30px;">There seem to be 7 rows where the results of `HeatU()` for the Double Triangle method with the intermediate and vertical cutoffs that differ from those from the IPM website</p>